# Posterior Distributions
Andrew Esch, Evan Lee, and Collin Stratton

CST-425

Dr. Isac Artzi

2 April, 2022

# Introduction
The purpose of this assignment is to explore Markov Chain Monte Carlo (MCMC) by predicting the likelihood of people visiting a given attraction. This project will analyze and visualize the posterior distributions of the locations visited by tourists.

# Setup

In [ ]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pymc3 as pm

from PosteriorDistributions import *

In [ ]:
# read in data
df = pd.read_csv("./syntheticdata.csv", header=None)
# row and column headers list
headers = ["Museum", "Concert", "Sports Event", "Restaurant", "Hike"]

print(df)

In [ ]:
# Convert into a probabilistic matrix, and then convert the probabilistic matrix into a transition matrix
transitionMatrix = df.div(df.sum(axis=1), axis=0)
transitionMatrix = transitionMatrix.to_numpy()

print(transitionMatrixNP)

# Markov Chain
## Generate Steady State Distribution Graphs from the Transition Matrix using Markov Chains

In [ ]:
# Generate graphs from transition matrix to illustrate the Markov Chain
for i in range(transitionMatrix.shape[0]):
    for j in range(transitionMatrix.shape[1]):
        markovPlot(transitionMatrix, i, j, 11)

## Finding Probabilities
### Find the likelihood that a tourist would attend a concert, then go on a hike, then eat at a restaurant

In [ ]:
# multiply the transition probability of concert to hike by hike to restaurant
concertHikeRestaurant = transitionMatrix[1][4] * transitionMatrix[4][3]
print("Percent odds of Concert to Hike to Restaurant = ", concertHikeRestaurant * 100)

### Find the likelihood of visiting any of the locations as the fifth step

In [ ]:
# take the transition matrix to the fifth power
trans5 = pow(transitionMatrix, 5)

# sum the columns of the fifth power matrix
sum_arr = trans5.sum(axis=0)

# print the likelihood of visiting any of the locations as the fifth step
print("\nPercent odds of visiting the Museum 5th = ", sum_arr[0] * 100)
print("Percent odds of visiting the Concert 5th = ", sum_arr[1] * 100)
print("Percent odds of visiting the Sports Event 5th = ", sum_arr[2] * 100)
print("Percent odds of visiting the Restaurant 5th = ", sum_arr[3] * 100)
print("Percent odds of visiting the Hike 5th = ", sum_arr[4] * 100)